# Setup

In [1]:
import importlib
import os
import sys

In [2]:
print(os.getcwd())

/workspaces/f33-gcp-mlops/examples/kaggle_store_item_demand_forecasting/models/xgb_ar


In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/workspaces/.gcp/waffleml_dev_key.json"

In [4]:
if not "model" in sys.modules:
    model = importlib.import_module("model", package=".")
    helpers = importlib.import_module("wml_helpers")
else:
    model = importlib.reload(model)
    pass

In [5]:
sys.argv = [
   "wml_wrapper.py",   
   "train",
   "--training_data_path","gs://f33-mlops/experiments/kaggle-store-item-demand-forecasting/folds/perf_1_hyper_2_train.csv",
   "--encoded_hyperparameters", "ml__n_estimators:int=10;ml__learning_rate:float=0.04060464247785402;ml__subsample:float=0.5833628685682365",
   "--export_path",  "gs://f33-mlops/experiments/kaggle-store-item-demand-forecasting/dev"
]

In [6]:
stage = helpers.parse_stage()
args = helpers.parse_arguments_by_stage(stage)

In [7]:
print("Downloading training data...")
trainig_data_local_path = helpers.get_filename_from_path(      
    args.training_data_path)

In [8]:
helpers.get_file_from_bucket(args.training_data_path,
                                trainig_data_local_path)
hyper_parameters = helpers.parse_hyper_parameters(
    args.encoded_hyperparameters)

In [12]:
print("Triggering training method ...")
artifact_local_path = model.train(trainig_data_local_path,
                                    hyper_parameters)
print("Finished!")

2022-10-19 20:12:29,697 - xgb_ar - INFO - Load & transform data...


Triggering training method ...


2022-10-19 20:12:46,541 - xgb_ar - INFO - done. Training model...


[0]	validation_0-mae:49.74206
[1]	validation_0-mae:47.72276
[2]	validation_0-mae:45.78588
[3]	validation_0-mae:43.92739
[4]	validation_0-mae:42.14384
[5]	validation_0-mae:40.43295
[6]	validation_0-mae:38.79173
[7]	validation_0-mae:37.21727
[8]	validation_0-mae:35.70619
[9]	validation_0-mae:34.25763


2022-10-19 20:12:51,664 - xgb_ar - INFO - done. Exporting model...
2022-10-19 20:12:51,798 - xgb_ar - INFO - done.


Finished!


In [12]:
if artifact_local_path is not None:
    print("Uploading generated artifact to Cloud Storage")
    artifact_bucket_path = helpers.get_output_bucket_file_path(
        stage, args.export_path)
    helpers.upload_file_to_bucket(artifact_local_path, artifact_bucket_path)
    print("Exiting.")
    print(artifact_bucket_path)

Uploading generated artifact to Cloud Storage
Exiting.
gs://f33-mlops/experiments/kaggle-store-item-demand-forecasting/dev/model.joblib


## Predict

In [13]:
sys.argv = [
   "wml_wrapper.py",
   "pred",
   "--model_path", "gs://f33-mlops/experiments/kaggle-store-item-demand-forecasting/dev/model.joblib",
   "--test_data_path", "gs://f33-mlops/experiments/kaggle-store-item-demand-forecasting/folds/perf_1_hyper_2_test.csv",
   "--export_path",  "gs://f33-mlops/experiments/kaggle-store-item-demand-forecasting/dev"
]

In [14]:
stage = helpers.parse_stage()
args = helpers.parse_arguments_by_stage(stage)

In [15]:
print("Downloading input files...")

# Download test data
test_data_local_path = helpers.get_filename_from_path(
    args.test_data_path)
helpers.get_file_from_bucket(args.test_data_path,
                             test_data_local_path)

In [16]:
# Download model
model_local_path = helpers.get_filename_from_path(args.model_path)
helpers.get_file_from_bucket(args.model_path, model_local_path)

In [17]:
print("Running prediciton ...")
artifact_local_path = model.predict(test_data_local_path,
                                    model_local_path)
print("Finished!")

2022-10-19 19:19:02,847 - xgb_ar - INFO - Import model...
2022-10-19 19:19:02,967 - xgb_ar - INFO - done. Load & transform data...


Running prediciton ...


2022-10-19 19:19:03,448 - xgb_ar - INFO - done. Generate predictions...
2022-10-19 19:19:03,535 - xgb_ar - INFO - done. Export predictions to csv file...
2022-10-19 19:19:03,632 - xgb_ar - INFO - done


Finished!


In [18]:
if artifact_local_path is not None:
    print("Uploading generated artifact to Cloud Storage")
    artifact_bucket_path = helpers.get_output_bucket_file_path(
        stage, args.export_path)
    helpers.upload_file_to_bucket(artifact_local_path, artifact_bucket_path)
    print("Exiting.")
    print(artifact_bucket_path) 

Uploading generated artifact to Cloud Storage
Exiting.
gs://f33-mlops/experiments/kaggle-store-item-demand-forecasting/dev/predictions.csv


## Eval

In [20]:
sys.argv = [
   "wml_wrapper.py",
   "eval",
   "--predictions_path", "gs://f33-mlops/experiments/kaggle-store-item-demand-forecasting/dev/predictions.csv",
   "--ground_truth_path", "gs://f33-mlops/experiments/kaggle-store-item-demand-forecasting/folds/perf_1_hyper_2_test.csv",
   "--export_path",  "gs://f33-mlops/experiments/kaggle-store-item-demand-forecasting/dev"
]

In [21]:
stage = helpers.parse_stage()
args = helpers.parse_arguments_by_stage(stage)

In [22]:
ground_truth_local_path = helpers.get_filename_from_path(
    args.ground_truth_path)
helpers.get_file_from_bucket(args.ground_truth_path,
                                ground_truth_local_path)

# Download predictions
predictions_local_path = helpers.get_filename_from_path(
    args.predictions_path)
helpers.get_file_from_bucket(args.predictions_path,
                                predictions_local_path)

print("Running evaluation ...")
artifact_local_path = model.evaluate(ground_truth_local_path,
                                        predictions_local_path)
print("Finished!")

Running evaluation ...
Finished!


In [ ]:
import yaml
from pathlib import Path

output_file="test.yaml"
with open(output_file, "w") as outfile:
    yaml.dump({ "mae":0.24, "mse":0.6}, outfile, default_flow_style=True)


metrics = yaml.safe_load(open("metrics.yaml"))
print(metrics)

In [ ]:
if artifact_local_path is not None:
    print("Uploading generated artifact to Cloud Storage")
    artifact_bucket_path = helpers.get_output_bucket_file_path(
        stage, args.export_path)
    helpers.upload_file_to_bucket(artifact_local_path, artifact_bucket_path)
    print("Exiting.") 